In [1]:
%matplotlib inline
%run ../../import_envs.py

K = 5 # number of digits
T = 100 # timesteps
## Model Parameters
FRAME_PIXELS = 96
DIGIT_PIXELS = 28
NUM_HIDDEN_DIGIT = 200
NUM_HIDDEN_COOR = 200
Z_WHERE_DIM = 2 # z_where dims
Z_WHAT_DIM = 10

MODEL_NAME = 'apg'
RESAMPLING_STRATEGY = 'systematic'
LOAD_VERSION = 'bmnist-3digits-apg-systematic-4sweeps-20samples-0.00020lr'
MNIST_MEAN_PATH = '../mnist_mean.npy'

CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:0')

probtorch: 0.0+5a2c637 torch: 1.3.0


In [2]:
# train data path
DATA_PATHS = []
TEST_DATA_DIR = '/data/hao/apg_data/bmnist/test/'
print('===Loading testing data from %s===' % TEST_DATA_DIR)
# for file in os.listdir(TEST_DATA_DIR):
#     DATA_PATHS.append(os.path.join(TEST_DATA_DIR, file))
    
DATA_PATHS.append(os.path.join(TEST_DATA_DIR, 'ob-1.npy'))
print('===%d groups of data files are loaded.===' % len(DATA_PATHS))

===Loading testing data from /data/hao/apg_data/bmnist/test/===
===1 groups of data files are loaded.===


In [3]:
from apg_modeling import init_model
from affine_transformer import Affine_Transformer

model = init_model(frame_pixels=FRAME_PIXELS,
                               digit_pixels=DIGIT_PIXELS, 
                               num_hidden_digit=NUM_HIDDEN_DIGIT, 
                               num_hidden_coor=NUM_HIDDEN_COOR, 
                               z_where_dim=Z_WHERE_DIM, 
                               z_what_dim=Z_WHAT_DIM, 
                               CUDA=CUDA, 
                               DEVICE=DEVICE, 
                               LOAD_VERSION=LOAD_VERSION, 
                               LR=None)
AT = Affine_Transformer(frame_pixels=FRAME_PIXELS, 
                        digit_pixels=DIGIT_PIXELS, 
                        CUDA=CUDA, 
                        DEVICE=DEVICE)

In [4]:
## visualize samples
%run ../hybrid_testing.py
APG_SWEEPS_TEST = 20
SAMPLE_SIZE_TEST = 10
BATCH_SIZE_TEST = 4
# DATA_PTR = 11
FLAGs = {'hmc' : True, 'apg' : False, 'bps' : False}

metrics = test_hybrid_all(model=model, 
                            flags=FLAGs,
                            AT=AT,
                            apg_sweeps=APG_SWEEPS_TEST, 
                            data_paths=DATA_PATHS, 
                            mnist_mean_path=MNIST_MEAN_PATH,
                            T=T,
                            K=K,
                            D=Z_WHERE_DIM,
                            z_what_dim=Z_WHAT_DIM,
                            batch_size=BATCH_SIZE_TEST,
                            sample_size=SAMPLE_SIZE_TEST, 
                            hmc_num_steps=APG_SWEEPS_TEST,
                            leapfrog_step_size=1e-4,
                            leapfrog_num_steps=20,
                            filename='test',
                            CUDA=CUDA, 
                            DEVICE=DEVICE)

# frames = sample_data_uniform(DATA_PATHS=DATA_PATHS, 
#                             data_ptr=DATA_PTR)

/home/hao/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2751: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "
/home/hao/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "


Running HMC updates..
1 / 250 in group 0 completed (785s)
Running HMC updates..


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-a6107436dd92>", line 26, in <module>
    DEVICE=DEVICE)
  File "/home/hao/Research/amortized/BMNIST/hybrid_testing.py", line 120, in test_hybrid_all
    leapfrog_num_steps=leapfrog_num_steps)
  File "/home/hao/Research/amortized/BMNIST/hybrid_objective.py", line 40, in hybrid_objective
    leapfrog_num_steps=leapfrog_num_steps)
  File "/home/hao/Research/amortized/BMNIST/hmc_objective.py", line 41, in hmc_sampling
    z_where, z_what = self.metrioplis(ob=ob, z_where=z_where.detach(), z_what=z_what.detach(), step_size_what=step_size_what, step_size_where=step_size_where, num_steps=leapfrog_num_steps)
  File "/home/hao/Research/amortized/BMNIST/hmc_objective.py", line 67, in metrioplis
    new_where, new_what, new_r_where, new_r_what = self.leapfrog(ob, z_where, z

KeyboardInterrupt: 

In [ ]:
# test_hybrid(model=model, 
#             flags=FLAGs,
#             AT=AT,
#             apg_sweeps=APG_SWEEPS_TEST, 
#             frames=frames, 
#             mnist_mean_path=MNIST_MEAN_PATH,
#             K=K,
#             D=Z_WHERE_DIM,
#             z_what_dim=Z_WHAT_DIM,
#             sample_size=SAMPLE_SIZE_TEST, 
#             hmc_num_steps=APG_SWEEPS_TEST,
#             leapfrog_step_size=1e-3,
#             leapfrog_num_steps=5,
#             filename='test',
#             CUDA=CUDA, 
#             DEVICE=DEVICE)

In [ ]:
import matplotlib.pyplot as plt
filename = 'test'
def plot_density(flags, fs, fs_title, lw):
    fig = plt.figure(figsize=(fs*3,fs)) 
    ax = fig.add_subplot(111)
    COLORS = ['#EE7733', '#009988', '#AA3377', '#0077BB', '#555555', '#999933']
    lw = lw
    fs_title= fs_title
    if flags['apg']:
        log_joint_apg = np.load('log_joint_apg_%s.npy' % filename)
        apg_mean = log_joint_apg.mean(1)
        apg_std = log_joint_apg.std(1)
        ax.plot(apg_mean, c=COLORS[0], label='APG')

    if flags['hmc']:
        log_joint_hmc = np.load('log_joint_hmc_%s.npy' % filename)
        hmc_mean = log_joint_hmc.mean(1)
        hmc_std = log_joint_hmc.std(1)
        ax.plot(hmc_mean, c=COLORS[1], label='HMC')

    if flags['bps']:
        log_joint_gibbs = np.load('log_joint_bps_%s.npy' % filename)
        gibbs_mean = log_joint_gibbs.mean(1)
        gibbs_std = log_joint_gibbs.std(1)
        ax.plot(gibbs_mean, c=COLORS[3], label='BPS')  
    ax.legend()
plot_density(flags=FLAGs, fs=8, fs_title=14, lw=2)

In [ ]:
filename = 'test'
log_joint_hmc = np.load('log_joint_hmc_%s.npy' % filename)
hmc_mean = log_joint_hmc.mean(1)

hmc_mean